# Oracles Tutorial

1) Motivation of the tutorial

2) Outline of the tutorial and what it sets out to accomplish

    Definitions of Quantum Oracles, including types of oracles
    
    Mathematical properties of Quantum Oracles
    https://github.com/microsoft/QuantumKatas/tree/main/tutorials/LinearAlgebra
    
    Demos of different quantum oracles
    
    phase kickback:
    https://quantumcomputing.stackexchange.com/questions/14206/should-c-not-gate-affect-input-qubit-or-not/14207#14207
    https://qiskit.org/textbook/ch-gates/phase-kickback.html
    
    Exercises to implement your own quantum oracles
    
    Final exercise to implement a more complex oracle as well as write tests for that oracle
    
3) Suggested tutorials or topics to be completed or familiar with before this tutorial – The Qubit (Tutorial), Linear Algebra (Tutorial), Basic Quantum Computing Gates (Tutorial), Superposition


# Part I - Introduction to Quantum Oracles

## Classical Oracles
In classical computing, the idea of white box versus black box testing is leveraged.  In white box testing, the implementation of the function, say to factor a number, is visable to the tester.  Thus the tester can test specific expectaions such as runtime or memory complexity.  However, in black box testing, the tester can only test the functionality and expected behavior of the function - the implementation has been abstracted away.

Generally, most functions and data structures that we use while programming are a black box in our eyes - we are not worried with how an linked list is implemented, we only care about the functionality that it provides.  

Suppose I provided you the following function:
the function takes two parameters as input, two lists, where these lists represent the availability during the week of two different employees at a company.  The function returns if true if there is a day (Monday, Tuesday, Wednesday, Thursday, or Friday) for which they are both free and could schedule a meeting, if no such date exists then the function returns false.

The previous proposition is an example of a **classical oracle** because when interacting with this function, you are not worried with *how* the function determines the answer, you only care about *what* the answer is when provided a specific input.

Formally, a **classical oracle** is a function for which when provided some input, it produces a specific *deterministic* output.  Hence the same input *always* results in the same output.

#### <span style="color:blue">Exercise 1</span>: Implement a classical oracle
Implement the following classical oracle:
* f(x) = 1 (True) if x = 7
* f(x) = 0 (False) otherwise
* provided that x is a binary string. Note, $x = 0101 = 5$, $x = 0110 = 6$, $x = 10001 = 17$

In [1]:
%kata E1_Classical_Oracle

function Is_Seven(x : String) : Bool {
    // ...
    return false;
}

Success!

## Quantum Oracles

An oracle in the quantum world is a black box operation that is utilized during an algorithm.  These **quantum oracles** are passed as input to quantum algorithms and they themselves require inputs to operate on.  A quantum oracle implements some function $f: \{0,1\}^n \rightarrow \{0,1\}^m$ where $x$ is the input state of the form $|x\rangle = |x_0\rangle \otimes |x_1\rangle \otimes ... \otimes |x_{n-1}\rangle$ There are two types of quantum oracles: phase oracles and marking oracles.  Each of these types oracles are defined based on their operations on their basis states, where these basis states are always $|0\rangle$ and $|1\rangle$.  Oracles must be unitary as well as not change the input if that input is a basis state of the oracle.  Oracles also follow the same rules of linear algebra as normal matricies as they themselves are matricies, refer to the intro to review some of the properties of quantum operators (matricies).

### Phase Oracles
A phase oracle $U_{phase}$ is an oracle that when provided some state $|x\rangle$ it flips the sign of that state if $f(x)=1$.  Thus:

$$U_{phase} |x\rangle = (-1)^{f(|x\rangle)}|x\rangle$$

#### <span style="color:blue">Demo 1</span>: Phase quantum oracle - alternating bitpattern!
Consider the following phase oracle $U_{alt}$:
* $U_{alt} |x\rangle = (-1)^{f(x)}|x\rangle$
* $f(x) = 1$ if $|x\rangle = |101\rangle$ or $|x\rangle = |010\rangle$
* $f(x) = 0$ otherwise
* $|x\rangle$ is composed of 3 qubits


In [21]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Diagnostics;

operation Phase_Oracle_Demo () : Unit {
    let divider = "--------------------------------------------------------------------------------------------------";
    
    // allocate the |000⟩ state
    using(q = Qubit[3]) {
        // Prepare an equal superposition of all basis states
        ApplyToEachA(H, q);
        
        // Print out that we currently have a superposition of all the basis states
        // Notice the phases on each of the components
        Message("State |β⟩ (equal superposition of all basis states):");
        DumpMachine();
        Message(divider);
        
        // Apply the oracle
        using(one = Qubit()) {
            within {
                X(one);
                for (i in IndexRange(q)) {
                    if (i % 2 == 0) {
                        X(q[i]);
                    }
                }
            } apply {
                Controlled Z(q, one);
            }
        }
        using(one = Qubit()) {
            within {
                X(one);
                for (i in IndexRange(q)) {
                    if (i % 2 == 1) {
                        X(q[i]);
                    }
                }
            } apply {
                Controlled Z(q, one);
            }
        }
        
        // Print out the resulting state of AND |β⟩
        // Notice that the phase of the state is still positive
        Message("State after applying the alternating phase oracle to |β⟩:");
        DumpMachine();
        Message(divider);
        
        // Reset our state q back to all zeros for deallocation
        ResetAll(q);
    }
}

Phase_Oracle_Demo

In [22]:
%config dump.basisStateLabelingConvention="Bitstring"
%simulate Phase_Oracle_Demo

State |β⟩ (equal superposition of all basis states):


|000⟩	0.35355339059327384 + 0𝑖
|001⟩	0.35355339059327384 + 0𝑖
|010⟩	0.35355339059327384 + 0𝑖
|011⟩	0.35355339059327384 + 0𝑖
|100⟩	0.35355339059327384 + 0𝑖
|101⟩	0.35355339059327384 + 0𝑖
|110⟩	0.35355339059327384 + 0𝑖
|111⟩	0.35355339059327384 + 0𝑖

--------------------------------------------------------------------------------------------------
State after applying the alternating phase oracle to |β⟩:


|000⟩	0.35355339059327384 + 0𝑖
|001⟩	0.35355339059327384 + 0𝑖
|010⟩	-0.35355339059327384 + 0𝑖
|011⟩	0.35355339059327384 + 0𝑖
|100⟩	0.35355339059327384 + 0𝑖
|101⟩	-0.35355339059327384 + 0𝑖
|110⟩	0.35355339059327384 + 0𝑖
|111⟩	0.35355339059327384 + 0𝑖

--------------------------------------------------------------------------------------------------


()

Now you will implement the same classical oracle that you implemented in <span style="color:blue">Exercise 1</span> as a quantum phase oracle.

#### <span style="color:blue">Exercise 2</span>: Implement a phase quantum oracle
Implement the following quantum oracle $U_{7,phase}$:
* $U_{7,phase} |x\rangle = (-1)^{f(x)}|x\rangle$
* $f(x) = 1$ if $x = 7$
* $f(x) = 0$ otherwise
* $|x\rangle$ is a state composted of $3$ qubits

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
  What internal state can you prepare such that $X|\psi\rangle = -|\psi\rangle$ or $Z|\psi\rangle = -|\psi\rangle$?  Finally use this internal state to flip the sign of the input state $|x\rangle$.
</details>

> **Don't allocate auxiliary qubits to preform this operation.**

In [2]:
%kata E2_Phase_Quantum_Oracle 

operation Phase_7_Oracle (x : Qubit[]) : Unit 
is Adj {
    // ...
    using (q = Qubit()) {
        X(q);
        
        Controlled Z(x, q);
        
        X(q);
    }
}

Success!

#### Mathematical properties

Consider how the oracle from <span style="color:blue">Exercise 2</span> acts on its basis states:
$$U_{7,phase} |111\rangle = -|111\rangle$$
$$U_{7,phase} |110\rangle = |110\rangle$$

This follows the requirement that a $U_{7,phase}$ does not change the input if it's a basis state as well as the fact that $U_{7,phase}$ does not change the norm of the state ($U_{7,phase}$ is unitary).  However, consider if we were provided a state in superposition instead, what might that look like?

Suppose that $|\beta\rangle$ is an equal super position of the $6$ and $7$ state: 
$$|\beta\rangle = \frac{1}{\sqrt{2}} \big(|110\rangle + |111\rangle\big) = |11\rangle \otimes \frac{1}{\sqrt{2}} \big(|0\rangle + |1\rangle\big) = |11\rangle \otimes |+\rangle = |11+\rangle$$

Lets consider how our operator acts on this new state:
$$U_{7,phase} |\beta\rangle = U_{7,phase} \Big[\frac{1}{\sqrt{2}} \big(|110\rangle + |111\rangle\big)\Big] = $$
$$= \frac{1}{\sqrt{2}} \big(U_{7,phase} |110\rangle + U_{7,phase} |111\rangle\big) = \frac{1}{\sqrt{2}} \big(|110\rangle - |111\rangle\big) := |\gamma\rangle$$

Now the question is if our input state was modified during this operation, lets simplify $|\gamma\rangle$:
$$|\gamma\rangle = \frac{1}{\sqrt{2}} \big(|110\rangle - |111\rangle\big) = |11\rangle \otimes \frac{1}{\sqrt{2}} \big(|0\rangle - |1\rangle\big) = $$
$$= |11\rangle \otimes |-\rangle = |11-\rangle \neq |\beta\rangle$$

Here we see that the oracle modifies the input.  This is conditioned on the input state being a *superposition* of the basis states of the oracle - as a phase oracle will only modify the sign of its basis states.

> It is also worth noting that while the oracle modified the input when provided a superposition state, it did *not* modify the norm of that state.  As an exercise, you can verify this yourself by taking the norm of $|\beta\rangle$ and $|\gamma\rangle$, which both will result in a value of $1$.
>
>
> As another exercise, consider how you could distinguish between the input and output state programatrically?  Is there an operation that you could apply to the initial state $|\beta\rangle$ and the final state $|\gamma\rangle$ to show that the two states are not equivalent?


### Marking Oracles

A marking oracle $U_{mark}$ is an oracle that when provided some state $|x\rangle$ and some qubit $y$ preforms addition modulo 2 between $f(x)$ and $y$.  Hence $U_{mark}$ is an operator that preforms the following operation:

$$U_{mark}|x\rangle |y\rangle = U_{mark}\big(|x\rangle \otimes |y\rangle\big) = |x\rangle \otimes |y \oplus f(x)\rangle = |x\rangle |y \oplus f(x)\rangle$$

#### <span style="color:blue">Demo 2</span>: Marking quantum oracle - alternating bitpattern!
Consider the following marking oracle $U_{alt}$:
* $U_{alt} |x\rangle|y\rangle = |x\rangle|y\oplus f(x)\rangle$
* $f(x) = 1$ if $x = |101\rangle$ or $|x\rangle = |010\rangle$
* $f(x) = 0$ otherwise
* $|x\rangle$ is a state composted of $3$ qubits
* $|y\rangle$ is a single qubit


In [24]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Diagnostics;

operation Marking_Oracle_Demo () : Unit {
    let divider = "--------------------------------------------------------------------------------------------------";
    
    // allocate the |000⟩|0⟩ state
    using(q = Qubit[4]) {
        // Prepare an unequal superposition of all basis states
        //PrepareArbitraryStateD(H, q[0..Length(q)-2]);
        ApplyToEachA(H, q[0..Length(q)-2]);
        
        // Print out that we currently have an unequal superposition of all the 
        // basis states.  Notice the phases on each of the components
        Message("State |β⟩ (equal superposition of all basis states):");
        DumpMachine();
        Message(divider);
        
        // Apply the oracle
        within {
            for (i in 0..Length(q)-2) {
                if (i % 2 == 0) {
                    X(q[i]);
                }
            }
        } apply {
            Controlled X(q[0..Length(q)-2], q[Length(q)-1]);
        }
        within {
            for (i in 0..Length(q)-2) {
                if (i % 2 == 1) {
                    X(q[i]);
                }
            }
        } apply {
            Controlled X(q[0..Length(q)-2], q[Length(q)-1]);
        }
        
        // Print out the resulting state of AND |β⟩
        // Notice that the phase of the state is still positive
        Message("State after applying the alterating marking oracle to |β⟩:");
        DumpMachine();
        Message(divider);
        
        // Reset our state q back to all zeros for deallocation
        ResetAll(q);
    }
}

Marking_Oracle_Demo

In [25]:
%config dump.basisStateLabelingConvention="Bitstring"
%simulate Marking_Oracle_Demo

State |β⟩ (equal superposition of all basis states):


|0000⟩	0.35355339059327384 + 0𝑖
|0001⟩	0 + 0𝑖
|0010⟩	0.35355339059327384 + 0𝑖
|0011⟩	0 + 0𝑖
|0100⟩	0.35355339059327384 + 0𝑖
|0101⟩	0 + 0𝑖
|0110⟩	0.35355339059327384 + 0𝑖
|0111⟩	0 + 0𝑖
|1000⟩	0.35355339059327384 + 0𝑖
|1001⟩	0 + 0𝑖
|1010⟩	0.35355339059327384 + 0𝑖
|1011⟩	0 + 0𝑖
|1100⟩	0.35355339059327384 + 0𝑖
|1101⟩	0 + 0𝑖
|1110⟩	0.35355339059327384 + 0𝑖
|1111⟩	0 + 0𝑖

--------------------------------------------------------------------------------------------------
State after applying the alterating marking oracle to |β⟩:


|0000⟩	0.35355339059327384 + 0𝑖
|0001⟩	0 + 0𝑖
|0010⟩	0.35355339059327384 + 0𝑖
|0011⟩	0 + 0𝑖
|0100⟩	0 + 0𝑖
|0101⟩	0.35355339059327384 + 0𝑖
|0110⟩	0.35355339059327384 + 0𝑖
|0111⟩	0 + 0𝑖
|1000⟩	0.35355339059327384 + 0𝑖
|1001⟩	0 + 0𝑖
|1010⟩	0 + 0𝑖
|1011⟩	0.35355339059327384 + 0𝑖
|1100⟩	0.35355339059327384 + 0𝑖
|1101⟩	0 + 0𝑖
|1110⟩	0.35355339059327384 + 0𝑖
|1111⟩	0 + 0𝑖

--------------------------------------------------------------------------------------------------


()

Now you will implement the same quantum oracle that you implemented in <span style="color:blue">Exercise 2</span> but this time as a marking oracle instead.

#### <span style="color:blue">Exercise 3</span>: Implement a marking quantum oracle
Implement the following quantum oracle $U_{7,mark}$:
* $U_{7,mark} |x\rangle |y\rangle = |x\rangle |y \oplus f(x)\rangle$
* $f(x) = 1$ if $x = 7$
* $f(x) = 0$ otherwise
* $|x\rangle$ is a state composted of $3$ qubits
* $|y\rangle$ is a single qubit

In [3]:
%kata E3_Marking_Quantum_Oracle 

operation Marking_7_Oracle(x: Qubit[], y: Qubit) : Unit
is Adj {
    // ...
    Controlled X(x, y);
}

Success!

#### Mathematical Properties

Again, let us consider how the marking oracle that you just implemented in <span style="color:blue">Exercise 3</span> affects the input when the input is the oracles basis states opposed to a superposition state.
$$U_{7,mark} |111\rangle |0\rangle = |111\rangle |0 \oplus f(111)\rangle = |111\rangle |0 \oplus 1\rangle = |111\rangle |1\rangle$$
$$U_{7,mark} |111\rangle |1\rangle = |111\rangle |1 \oplus f(111)\rangle = |111\rangle |1 \oplus 1\rangle = |111\rangle |0\rangle$$

$$U_{7,mark} |110\rangle |0\rangle = |110\rangle |0 \oplus f(110)\rangle = |110\rangle |0 \oplus 0\rangle = |110\rangle |0\rangle$$
$$U_{7,mark} |110\rangle |1\rangle = |110\rangle |1 \oplus f(110)\rangle = |110\rangle |1 \oplus 0\rangle = |110\rangle |1\rangle$$

In the previous examples, all of the input $|x\rangle$ and $|y\rangle$ are in basis states of the oracle $U_{mark}^7$, however we see that even when providing input states, if $f(x)=1$ we will change the value of our input $|y\rangle$.  However, if $f(x)=0$ then neither the input $|x\rangle$ or $|y\rangle$ will be modified by the oracle.  By definition, if the marking oracle evaluates to one for some input, hence if $f(x)=1$ for some $x$, then the input state $|y\rangle$ will be changed.  Specifically, $|y\rangle$ will be flipped.

Now let us define a state $|\alpha\rangle$ such that $|x\rangle$ is a superposition of the $6$ and $7$ basis states and $|y\rangle = |0\rangle$:
$$|\alpha\rangle = \frac{1}{\sqrt{2}}\big(|110\rangle |0\rangle + |111\rangle |0\rangle\big) = $$
$$= |11\rangle \otimes \frac{1}{\sqrt{2}} \big(|0\rangle + |1\rangle\big) \otimes |0\rangle = |11+\rangle |0\rangle$$

How does our marking oracle $U_{7,mark}$ act on our state $|\alpha\rangle$?
> Recall that oracles are simply operators and operators are linear thus they can be distrubuted to each term individually.

$$U_{7,mark} |\alpha\rangle = \frac{1}{\sqrt{2}} \big(U_{7,mark}|110\rangle |0\rangle + U_{7,mark}|111\rangle |0\rangle\big) =$$
$$= \frac{1}{\sqrt{2}} \big(|110\rangle |0\rangle + |111\rangle |1\rangle\big) := |\epsilon\rangle$$

Now we would like to observe how our input state $|\alpha\rangle$ was modified by the oracle.  Let's simplify the resulting state from the oracle, $|\epsilon\rangle$:
$$|\epsilon\rangle = \frac{1}{\sqrt{2}} \big(|110\rangle |0\rangle + |111\rangle |1\rangle\big) = |11\rangle \otimes \frac{1}{\sqrt{2}} \big(|0\rangle |0\rangle + |1\rangle |1\rangle\big) = $$
$$= |11\rangle \otimes \frac{1}{\sqrt{2}} \big(|00\rangle + |11\rangle\big) = |11\rangle \otimes |\Phi^+\rangle = |11\Phi^+\rangle$$

Here we see that we have now entangled our input states $|x\rangle$ and $|y\rangle$!  This is a common occurance for marking oracles when the input is a superposition of basis states: after applying the oracle, the input $|x\rangle$ will become entangled with $|y\rangle$.  Here we see that the result is the bell state $|\Phi^+\rangle$.

>As an exercise, what entangled state would we get in the previous example if $|y\rangle = |1\rangle$ instead of $|y\rangle = |0\rangle$?
>
> <br/>
> <details>
>   <summary><b>Click here for the answer!</b></summary>
>   The entangled state that is produced would be $|\Psi^+\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)$.  Overall, we would get the following: $U_{mark}^7 |011+\rangle |1\rangle = |011\rangle |\Psi^+\rangle$
> </details>

# Part II: Phase Kickback

Previously with marking oracles we considered when the register $|x\rangle$ was in a basis state or a superposition state with the target qubit $|y\rangle$ in a basis state $\big( |0\rangle \text{ or } |1\rangle \big)$.  However, how might the affect of our marking oracles change if our target is *also* in a superposition state?  This case is where we will to observe **phase kickback**.

**Phase kickback** is a very important occurance in quantum computing and is a key ingredient to many quantum algorithms.  When we apply a marking oracle on a register $|x\rangle$ which is a superposition of state with a target qubit $|y\rangle$ which is also in superposition, the phase from $|y\rangle$ will be *kicked back* into our register $|x\rangle$.  

In order to observe phase kickback, we must use the target qubit $|y\rangle=|-\rangle$.  This is the standard choice for two reasons.  First, for phase kickback to occur and be observable, the target qubit must have a difference in phase between the two basis states $|0\rangle$ and $|1\rangle$.  Second, $|y\rangle$ must be an equal superposition, if it's not then the target $|y\rangle$ and the register $|x\rangle$ will become entangled after applying the marking oracle.

Generally, if we have a markig oracle $U_{mark}$ which implements some function $f(x)$ and we apply this marking oracle on the register $|x\rangle$ and target qubit $|y\rangle=|-\rangle$ we will have the following results:
* if $|x\rangle$ is in a basis state:
$$U_{mark} |x\rangle |y\rangle = U_{mark} |x\rangle |-\rangle = (-1)^{f(x)}|x\rangle |-\rangle = (-1)^{f(x)}|x\rangle |y\rangle$$


* if $|x\rangle$ is in a superposition state, say $|x\rangle = \frac{1}{\sqrt{3}} \big(|b_1\rangle + |b_2\rangle + |b_3\rangle \big)$ where $|b_1\rangle, |b_2\rangle$ and $|b_3\rangle$ are the basis states of $U_{mark}$ then:
$$U_{mark} |x\rangle |y\rangle = U_{mark} |x\rangle |-\rangle = U_{mark} \frac{1}{\sqrt{3}} \big(|b_1\rangle + |b_2\rangle + |b_3\rangle \big) |-\rangle = $$
$$ = \frac{1}{\sqrt{3}} \big( U_{mark}|b_1\rangle + U_{mark}|b_2\rangle + U_{mark}|b_3\rangle\big) |-\rangle = $$
$$ = \frac{1}{\sqrt{3}} \big( (-1)^{f(b_1)}|b_1\rangle + (-1)^{f(b_2)}|b_2\rangle + (-1)^{f(b_3)}|b_3\rangle\big) |-\rangle = $$
$$ =\frac{1}{\sqrt{3}} \big( (-1)^{f(b_1)}|b_1\rangle + (-1)^{f(b_2)}|b_2\rangle + (-1)^{f(b_3)}|b_3\rangle\big) |y\rangle$$

From these two cases we see that in the end, $U_{mark}$ does not change the target qubit $|y\rangle$ while it does change the register $|x\rangle$ thus we can drop the target qubit $|y\rangle$ without any repercussions.  Notice that if we do indeed drop the target qubit, we now have the following state:
$$|\psi\rangle = \frac{1}{\sqrt{3}} \big( (-1)^{f(b_1)}|b_1\rangle + (-1)^{f(b_2)}|b_2\rangle + (-1)^{f(b_3)}|b_3\rangle\big)$$

Which looks as if we had just applied a phase oracle to it instead of a marking oracle.  This is one very important application of phase estimation: it allows us to convert between a marking oracle and a phase oracle - which you will implement in the next exercise!  Another important application is phase estimation **(@William: go into a little more detail on phase estimation)**

To become more familiar with the mathematics of phase estimation we will consider the following example using the oracle you previously implemented: $U_{7,mark}$. Consider beginning with $|x\rangle$ as an equal superposition of the $6$ and $7$ basis states and $|y\rangle=|-\rangle$, our overall state $|\eta\rangle$ is the following:
$$|\eta\rangle = \Big[\frac{1}{\sqrt{2}}\big(|110\rangle + |111\rangle\big)\Big] \otimes \frac{1}{\sqrt{2}}\big(|0\rangle - |1\rangle\big) = $$
$$ = \frac{1}{2} \big(|110\rangle|0\rangle + |111\rangle|0\rangle - |110\rangle|1\rangle - |111\rangle|1\rangle\big)$$

How does $U_{7,mark}$ act on our state $|\eta\rangle$?
$$U_{7,mark}|\eta\rangle = U_{7,mark} \frac{1}{2} \big(|110\rangle|0\rangle + |111\rangle|0\rangle - |110\rangle|1\rangle - |111\rangle|1\rangle \big) = $$
$$= \frac{1}{2} \big( U_{7,mark}|110\rangle|0\rangle + U_{7,mark}|111\rangle|0\rangle - U_{7,mark}|110\rangle|1\rangle - U_{7,mark}|111\rangle|1\rangle \big) = $$
$$= \frac{1}{2} \big(|110\rangle|0\rangle + |111\rangle|1\rangle - |110\rangle|1\rangle - |111\rangle|0\rangle \big) := |\xi\rangle$$

Now we would like to observe how our input state $|\eta\rangle$ was modified by the oracle.  Let's simplify the resulting state from the oracle, $|\xi\rangle$:
$$|\xi\rangle = \frac{1}{2} \big(|110\rangle|0\rangle + |111\rangle|1\rangle - |110\rangle|1\rangle - |111\rangle|0\rangle\big)  = $$
$$= \frac{1}{2} \big(|110\rangle|0\rangle - |110\rangle|1\rangle - |111\rangle|0\rangle + |111\rangle|1\rangle \big) = $$
$$= \frac{1}{2} \Big[|110\rangle \otimes \big(|0\rangle - |1\rangle \big) + |111\rangle \otimes \big(|1\rangle - |0\rangle\big)\Big] = $$
$$ = \Big[\frac{1}{\sqrt{2}} \big( |110\rangle - |111\rangle \big) \Big] \otimes \Big[ \frac{1}{\sqrt{2}} \big( |0\rangle - |1\rangle \big) \Big] = $$
$$= \Big[\frac{1}{\sqrt{2}} \big( |110\rangle - |111\rangle \big) \Big] \otimes |-\rangle$$

Finally lets compare $|\eta\rangle$ and $|\xi\rangle$ directly, below the final equations are repeated for your convinence:
$$|\eta\rangle = \Big[\frac{1}{\sqrt{2}}\big(|110\rangle + |111\rangle\big)\Big] \otimes |-\rangle$$
$$|\xi\rangle = \Big[\frac{1}{\sqrt{2}}\big(|110\rangle - |111\rangle\big)\Big] \otimes |-\rangle$$

We can see that these two equations are identical except for the phase on the $|111\rangle$ basis state (representing $7$).  This is a specific example where phase kickback occurs as the phase from $|y\rangle$ has been *kicked back* into $|x\rangle$.

How could we determine wether or not phase kickback had occured in the previous example?  Consider the following simplification of our final equations first:
$$|\eta\rangle = \Big[\frac{1}{\sqrt{2}}\big(|110\rangle + |111\rangle\big)\Big] \otimes |-\rangle =$$
$$= |11\rangle \otimes \frac{1}{\sqrt{2}}\big( |0\rangle + |1\rangle \big) \otimes |-\rangle = |11+\rangle |-\rangle$$


$$|\xi\rangle = \Big[\frac{1}{\sqrt{2}}\big(|110\rangle - |111\rangle\big)\Big] \otimes |-\rangle =$$
$$= |11\rangle \otimes \frac{1}{\sqrt{2}}\big( |0\rangle - |1\rangle \big) \otimes |-\rangle = |11-\rangle |-\rangle$$

<br/>
<details>
  <summary><b>Now how could we differentiate between the state $|11+\rangle |-\rangle$ and $|11-\rangle |-\rangle$?  Take a moment to think, then click here to see if you were correct!</b></summary>
  
    That's right!  If we apply a Hadamard operator to the third qubit, we will be able to distinguish between the input state and the output state.
    
    $$(I\otimes I \otimes H)|11+\rangle = |110\rangle$$
    $$(I\otimes I \otimes H)|11-\rangle = |111\rangle$$
    
    Now if we were to measure the input state versus the output state we would detect that the phase from $|y\rangle=|-\rangle$ was kicked back into our input state $|x\rangle$!
</details>


Now we will explore how you could create a oracle converter

#### <span style="color:blue">Exercise 4</span>: Apply the phase oracle
* **Input:** 
    * a marking oracle
    * a list of qubits to apply the phase oracle with
* After execution, the list of qubits will have had a phase oracle applied to them, where this phase oracle implements the same function $f(x)$ as the marking oracle does.

> A oracle converter is useful because many quantum algorithms rely on a phase oracle, such as Grover's algorithm, however it is often easier to implement a marking oracle.  This converter will provide us a way to convert a marking oracle we are interested in into a phase oracle; which could then be leveraged in a quantum algorithm such as Grover's algorithm.

<br/>
<details>
  <summary><b>If you're stuck on where to begin, click here!</b></summary>
    Recall that you can allocate extra qubits to assist in any operation.  Is there a state that you could prepare with an auxiliary qubit which would enable you to flip the phase of the input state $|x\rangle$ subject to the function $f(x)$ for the marking oracle?
</details>

In [5]:
%kata E4_Apply_Phase_Oracle

//operation Apply_Phase_Oracle(markingOracle: ((Qubit[], Qubit) => Unit is Adj), qubits: Qubit[]) : Unit
//is Adj {
    // ...
//}

operation Apply_Phase_Oracle(markingOracle: ((Qubit[], Qubit) => Unit is Adj), qubits: Qubit[]) : Unit
is Adj {
    using (minus = Qubit()) {
        // within - apply
        // this block tells you to first do the within block, then do the apply
        // then do the adjoint of the within block.
        within {
            X(minus);
            H(minus);
        } apply {
            markingOracle(qubits, minus);
        }
    }
}

Apply_Phase_Oracle

#### <span style="color:blue">Demo 3</span>: Oracle Converter
* **Input:**
    * a marking oracle
* **Output:**
    * a phase oracle which takes a register as input and flips the phase of that register according to the function $f(x)$ that the marking oracle implements

In this demo we will use the oracle converter to convert your previously implemented marking oracle, $U_{7,mark}$ into a phase oracle.

In [10]:
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Diagnostics;

function Oracle_Converter(markingOracle: ((Qubit[], Qubit) => Unit is Adj)) : (Qubit[] => Unit is Adj) {
    return Apply_Phase_Oracle(markingOracle, _);
}

operation Oracle_Converter_Demo() : Unit {
    let divider = "--------------------------------------------------------------------------------------------------";

    // allocate the state |000⟩
    using (register = Qubit[3]) {
        // convert to the equal superposition state
        ApplyToEachA(H, register);
        
        // dump the state
        Message("the equal superposition register before applying the phase oracle you implemented:");
        DumpMachine();
        Message(divider);
        
        // apply the oracle you implemented:
        Phase_7_Oracle(register);
        
        // dump the state after application of the oracle
        Message("the equal superposition register after applying the phase oracle you implemented:");
        DumpMachine();
        Message(divider);
        
        // reset the qubits for deallocation
        ResetAll(register);
    }
    
    // allocate the state |000⟩
    using (register = Qubit[3]) {
        // convert to the equal superposition state
        ApplyToEachA(H, register);
        
        // dump the state
        Message("the equal superposition register before applying the converted marking oracle:");
        DumpMachine();
        Message(divider);
        
        // convert the marking oracle
        let Converted_7_Oracle = Oracle_Converter(Marking_7_Oracle);
        
        // apply the converted marking oracle
        Converted_7_Oracle(register);
        
        // dump the state after application of the oracle
        Message("the equal superposition register after applying the converted marking oracle:");
        DumpMachine();
        Message(divider);
        
        // reset the qubits for deallocation
        ResetAll(register);
    }
}

Oracle_Converter, Oracle_Converter_Demo

In [11]:
%config dump.basisStateLabelingConvention="Bitstring"
%simulate Oracle_Converter_Demo

the equal superposition register before applying the phase oracle you implemented:


|000⟩	0.35355339059327384 + 0𝑖
|001⟩	0.35355339059327384 + 0𝑖
|010⟩	0.35355339059327384 + 0𝑖
|011⟩	0.35355339059327384 + 0𝑖
|100⟩	0.35355339059327384 + 0𝑖
|101⟩	0.35355339059327384 + 0𝑖
|110⟩	0.35355339059327384 + 0𝑖
|111⟩	0.35355339059327384 + 0𝑖

--------------------------------------------------------------------------------------------------
the equal superposition register after applying the phase oracle you implemented:


|000⟩	0.35355339059327384 + 0𝑖
|001⟩	0.35355339059327384 + 0𝑖
|010⟩	0.35355339059327384 + 0𝑖
|011⟩	0.35355339059327384 + 0𝑖
|100⟩	0.35355339059327384 + 0𝑖
|101⟩	0.35355339059327384 + 0𝑖
|110⟩	0.35355339059327384 + 0𝑖
|111⟩	-0.35355339059327384 + 0𝑖

--------------------------------------------------------------------------------------------------
the equal superposition register before applying the converted marking oracle:


|000⟩	0.35355339059327384 + 0𝑖
|001⟩	0.35355339059327384 + 0𝑖
|010⟩	0.35355339059327384 + 0𝑖
|011⟩	0.35355339059327384 + 0𝑖
|100⟩	0.35355339059327384 + 0𝑖
|101⟩	0.35355339059327384 + 0𝑖
|110⟩	0.35355339059327384 + 0𝑖
|111⟩	0.35355339059327384 + 0𝑖

--------------------------------------------------------------------------------------------------
the equal superposition register after applying the converted marking oracle:


|000⟩	0.35355339059327384 + 0𝑖
|001⟩	0.35355339059327384 + 0𝑖
|010⟩	0.35355339059327384 + 0𝑖
|011⟩	0.35355339059327384 + 0𝑖
|100⟩	0.35355339059327384 + 0𝑖
|101⟩	0.35355339059327384 + 0𝑖
|110⟩	0.35355339059327384 + 0𝑖
|111⟩	-0.35355339059327384 + 0𝑖

--------------------------------------------------------------------------------------------------


()

# Part III: Implementing Quantum Oracles

In this section you will implement a few quantum oracles of your own - each representing a specified function.  If you observe the operation declarations below, you will see that there is a requirement for the adjoint of the operation to be valid - the operation preformed in reverse order must be valid.  This is common practice: when we write quantum oracles, we want to undo any operations we may have performed on the input to achive the result in order to ensure that the oracle does not change the input if that input is a basis state of the oracle.

#### <span style="color:blue">Exercise 5</span>: Implement the OR oracle
Implement the following quantum oracle $U_{or}$:
* $U_{or} |x\rangle |y\rangle = |x\rangle |y \oplus f(x)\rangle$
* $f(x) = 0$ if $|x\rangle=|000...0\rangle$ (all zeros)
* $f(x) = 1$ otherwise
* $|x\rangle$ is a state composted of $n$ qubits
* $|y\rangle$ is a single qubit

<br/>
<details>
  <summary><b>Before implementing this oracle, answer the following question: are you implmenting a marking or a phase oracle?  Click here for the answer!</b></summary>
    Correct!  This is a marking oracle because we are flipping the value of some target qubit $|y\rangle$ based on the value of the other input $|x\rangle$
</details>

In [29]:
$kata E5_Or_Oracle

operation Or_Oracle(x: Qubit[], y: Qubit) : Unit
is Adj {
    // ...
}

C:\snippet_.qs(1,1): error QS3001: Syntax does not match any known patterns.


#### <span style="color:blue">Exercise 6</span>: Implement the kth spin-up oracle:
Implement the following quantum oracle $U_{k,spinup}$:
* $U_{k,spinup} |x\rangle = (-1)^{f(x)}|x\rangle$
* $f(x) = 1$ if the $k$th qubit in $|x\rangle$ is a $1$ $\big(\text{if } |x\rangle_k=|1\rangle \big)$
* $f(x) = 0$ otherwise
* $|x\rangle$ is a state composted of $n$ qubits
* $k$ is an integer with $k\in \{0, ..., n-1 \}$

<br/>
<details>
  <summary><b>Before implementing this oracle, answer the following question: are you implmenting a marking or a phase oracle?  Click here for the answer!</b></summary>
    Correct!  This is a phase oracle because we are applying a phase to the input state $|x\rangle$ based on the evaluation of the function $f(x)$.
</details>

> Can you implement this function without using auxiliary qubits?

In [28]:
%kata E6_kth_Spin_Up

operation kth_Spin_Up(x: Qubit[], k: Int) : Unit 
is Adj {
    // ...
}

Success!

As you noticed in the previous exericse, undoing operations preformed on the input is essential to ensure that we are implementing oracles that follow the requirements of an oracle - oracles are unitary and they do not change the input if that input is a basis state.  Another key tool to have when implementing quantum oracles is allocation of auxiliary qubits to assist in a computation.  Below are a some exercises where you will practice allocating extra qubits to assist with the computation of $f(x)$.  **(@William:  Move this to the next section?)**

#### <span style="color:blue">Exercise 7</span>: Implement the kth qubit exclused OR oracle
Implement the following quantum oracle $U_{k,or}$:
* $U_{k,or} |x\rangle = (-1)^{f(x)}|x\rangle$
* $f(x) = 1$ if $x\rangle$ with the kth qubit satisfies the OR oracle condition
* $f(x) = 0$ otherwise
* $|x\rangle$ is a state composted of $n$ qubits

<br/>
<details>
  <summary><b>Before implementing this oracle, answer the following question: are you implmenting a marking or a phase oracle?  Click here for the answer!</b></summary>
    Correct!  This is a phase oracle because we are applying a phase to the input state $|x\rangle$ based on the evaluation of the function $f(x)$.
</details>

In [13]:
%kata E7_kth_Excluded_Or

operation kth_Excluded_Or(x: Qubit[], k: Int) : Unit
is Adj {
    // ...
}

Success!

consider your implemntations of the marking and phase oracle this shows that they are identical via demo 3

In [ ]:
// demo this

// write test cases for the alternating phase oracle
// compare this implementation with the oracle produced when 
// providing the marking oracle as input to the oracle converter 
// so the learner can convince themselves of their oracle converter
// implementation

# Part IV: More Oracles!  Implementation and Testing:

**teach them to allocate aux qubits for computation**

#### <span style="color:blue">Exercise 8</span>: Implement the arbitrary bitpattern oracle
Implement the following quantum oracle $U_{arbpat}$:
* $U_{arbpat} |x\rangle |y\rangle = |x\rangle |y \oplus f(x)\rangle$
* $f(x) = 1$ if $x\rangle$ follows the pattern outlined in the input list of booleans
* $f(x) = 0$ otherwise
* $|x\rangle$ is a state composted of $n$ qubits
* b is a list of n booleans where False indicates a zero should appear and True indicates a one
    * example: [False, True, False] $\rightarrow |010\rangle = |x\rangle$

In [ ]:
%kata E8_Arbitrary_Pattern_Oracle

operation Arbitrary_Pattern_Oracle(x: Qubit[], y: Qubit, b: Bool[]) : Unit 
is Adj {
    // ...
}

In [ ]:
// bonus challenge task: same as above but as a phase oracle
    // however they cannot allocate extra qubits.

#### <span style="color:blue">Exercise 9</span>: Implement an SAT style oracle

Suppose that you would like to schedule a meeting with your co-worker Jasmine.  $|x\rangle$ represents your schedule for the five day workweek and $|jasmine\rangle$ represents Jasmine's schedule.  If a $1$ appears in either schedule it means that respective person is busy that day, a $0$ means they're free for a meeting that day.  Implement an oracle that determines if yourself and Jasmine can schedule a meeting.

Implement the following quantum oracle $U_{SAT}$:
* $U_{SAT} |x\rangle |jasmine\rangle |z\rangle = |x\rangle |jasmine\rangle |z \oplus f(x, jasmine)\rangle$
* $f(x, jasmine) = 1$ if $x\rangle$ and $|y\rangle$ encode a free day for both schedules
* $f(x, jasmine) = 0$ otherwise
* $|x\rangle$ and $|jasmine\rangle$ are states composted of $5$ qubits representing Monday, Tuesday, Wednesday, Thursday, and Friday
* $|z\rangle$ is a single qubit
    
**(@William) Do the is this a marking or phase oracle question!**

In [ ]:
%kata E9_Meeting_Oracle

operation Meeting_Oracle(x: Qubit[], jasmine: Qubit[], z: Qubit) : Unit 
is Adj {
    // ...
}

In [ ]:
// provide a demo of how you could test the oracle
// so that they know how they could test their code in industry

# Part V: What's next?